# **Bitcoin price forecasting - Random Forest**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



# Dependencies, Libraries and Tools

In [1]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
SLOW_OPERATION = False

In [61]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
#Install some useful dependencies
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from itertools import cycle

import plotly.express as px

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import gc

import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

# !pip install -U -q PyDrive # To use files that are stored in Google Drive directly (e.g., without downloading them from an external URL)
# !apt install openjdk-8-jdk-headless -qq
# import os
# os.environ["JAVA_HOME"] = JAVA_HOME

In [3]:
# Install Spark and related dependencies
!pip install pyspark

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml import Pipeline

# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPriceForecasting").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=0ec99257c309c576c04432f5e7a6486570c2c74177a3e514673ed729aa37d448
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


# Link to Google Drive

In [9]:
# Define GDrive paths
GDRIVE_DIR = "/content/drive"

GDRIVE_DATASET_OUTPUT_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/output"

GDRIVE_DATASET_NAME = "bitcoin_blockchain_data_30min"
GDRIVE_DATASET_NAME_TEST = GDRIVE_DATASET_NAME + "_test"

GDRIVE_DATASET_NAME_EXT_TEST = "/" + GDRIVE_DATASET_NAME_TEST + ".parquet"

GDRIVE_DATASET_TEST = GDRIVE_DATASET_OUTPUT_DIR + GDRIVE_DATASET_NAME_EXT_TEST

In [5]:
# Point Colaboratory to our Google Drive
from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


# Testing the models ❗

In [31]:
# Load datasets into pyspark dataframe objects
test_df = spark.read.load(GDRIVE_DATASET_TEST,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [23]:
PATH_MODELS = "/content/drive/MyDrive/BDC/project/models/"

LR_MODEL_NAME = "LinearRegression"
GLR_MODEL_NAME = "GeneralizedLinearRegression"
RF_MODEL_NAME = "RandomForestRegressor"
GBT_MODEL_NAME = "GBTRegressor"

In [21]:
GDRIVE_FEATURES_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/features"

GDRIVE_ALL_FEATURES_NAME = "all_features"
GDRIVE_COR_MATRIX_FEATURES_NAME = "cm_features"

GDRIVE_ALL_FEATURES_NAME_EXT = "/" + GDRIVE_ALL_FEATURES_NAME + ".json"
GDRIVE_COR_MATRIX_FEATURES_NAME_EXT = "/" + GDRIVE_COR_MATRIX_FEATURES_NAME + ".json"

GDRIVE_ALL_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_ALL_FEATURES_NAME_EXT
GDRIVE_COR_MATRIX_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_COR_MATRIX_FEATURES_NAME_EXT

In [13]:
# Loading all features
with open(GDRIVE_ALL_FEATURES, "r") as f:
    all_features = json.load(f)
print(all_features)

# Loading correlation matrix features
with open(GDRIVE_COR_MATRIX_FEATURES, "r") as f:
    cm_features = json.load(f)
print(cm_features)

# Set the depended variable
TARGET_VAL = 'tomorrow-market-price'

# Set the features label
FEATURES_LABEL = "features"

['total-bitcoins', 'market-cap', 'trade-volume', 'blocks-size', 'avg-block-size', 'n-transactions-total', 'n-transactions-per-block', 'hash-rate', 'difficulty', 'miners-revenue', 'transaction-fees-usd', 'n-unique-addresses', 'n-transactions', 'estimated-transaction-volume-usd']
['market-cap', 'miners-revenue', 'estimated-transaction-volume-usd', 'n-transactions-total', 'blocks-size', 'total-bitcoins', 'difficulty']


In [24]:
# Upload the 3 templates saved in Google Drive
from pyspark.ml import PipelineModel

lr = PipelineModel.load(PATH_MODELS + LR_MODEL_NAME)
glr = PipelineModel.load(PATH_MODELS + GLR_MODEL_NAME)
rf = PipelineModel.load(PATH_MODELS + RF_MODEL_NAME)
gbt = PipelineModel.load(PATH_MODELS + GBT_MODEL_NAME)

In [47]:
from sklearn.metrics import mean_absolute_percentage_error

def test_final_model(dataframe, trained_model, ml_model, label_col):
  # Make predictions
  predictions = trained_model.transform(dataframe)

  results = []

  # Compute validation error by several evaluator
  rmse_evaluator = RegressionEvaluator(labelCol=label_col, predictionCol="prediction", metricName='rmse')
  mae_evaluator = RegressionEvaluator(labelCol=label_col, predictionCol="prediction", metricName='mae')
  r2_evaluator = RegressionEvaluator(labelCol=label_col, predictionCol="prediction", metricName='r2')
  var_evaluator = RegressionEvaluator(labelCol=label_col, predictionCol="prediction", metricName='var')

  predictions_pd = predictions.select(label_col,"prediction").toPandas()
  mape = mean_absolute_percentage_error(predictions_pd[label_col], predictions_pd["prediction"])

  rmse = rmse_evaluator.evaluate(predictions)
  mae = mae_evaluator.evaluate(predictions)
  var = var_evaluator.evaluate(predictions)
  r2 = r2_evaluator.evaluate(predictions)
  # Adjusted R-squared
  n = predictions.count()
  p = len(predictions.columns)
  adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

  # Use dict to store each result
  results = {
      "Model": ml_model,
      "RMSE": rmse,
      "MAPE": mape,
      "MAE": mae,
      "Variance": var,
      "R2": r2,
      "Adjusted_R2": adj_r2,
  }

  return results, predictions.toPandas()

In [37]:
def select_features(dataset, features, featureCol, labelCol):
  vectorAssembler = VectorAssembler(
    inputCols = features,
    outputCol = featureCol)

  dataset = vectorAssembler.transform(dataset)
  dataset = dataset.select(['timestamp','id', featureCol, labelCol])

  return dataset

In [48]:
test_df_cm = select_features(test_df, cm_features, FEATURES_LABEL, TARGET_VAL)

lr_results, lr_predictions = test_final_model(test_df_cm, lr, LR_MODEL_NAME,"tomorrow-market-price")
glr_results, glr_predictions = test_final_model(test_df_cm, glr, GLR_MODEL_NAME, "tomorrow-market-price")
rf_results, rf_predictions = test_final_model(test_df_cm, rf, RF_MODEL_NAME, "tomorrow-market-price")
gbt_results, gbt_predictions = test_final_model(test_df_cm, gbt, GBT_MODEL_NAME, "tomorrow-market-price")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning:

Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning:

Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning:

Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning:

Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead



In [49]:
lr_results

{'Model': 'LinearRegression',
 'RMSE': 503.7448756461039,
 'MAPE': 0.013368366328303122,
 'MAE': 374.734297223603,
 'Variance': 2703575.1928723436,
 'R2': 0.9000886445359538,
 'Adjusted_R2': 0.9000133533863954}

In [50]:
glr_results

{'Model': 'GeneralizedLinearRegression',
 'RMSE': 503.7448756461039,
 'MAPE': 0.013368366328303122,
 'MAE': 374.734297223603,
 'Variance': 2703575.1928723436,
 'R2': 0.9000886445359538,
 'Adjusted_R2': 0.9000133533863954}

In [51]:
rf_results

{'Model': 'RandomForestRegressor',
 'RMSE': 3813.90444194934,
 'MAPE': 0.11725595669192056,
 'MAE': 3370.800172445324,
 'Variance': 13616833.254170353,
 'R2': -4.727079125438967,
 'Adjusted_R2': -4.73139493487788}

In [52]:
gbt_results

{'Model': 'GBTRegressor',
 'RMSE': 3147.423215899413,
 'MAPE': 0.09686337757948689,
 'MAE': 2715.0914271627826,
 'Variance': 13918441.583704391,
 'R2': -2.900352476501266,
 'Adjusted_R2': -2.903291702180619}

In [64]:
def show_results(test_df, lr_predictions, glr_predictions, rf_predictions, gbt_predictions):
  trace1 = go.Scatter(
      x = test_df['timestamp'],
      y = test_df['market-price'].astype(float),
      mode = 'lines',
      name = 'Test set'
  )

  trace2 = go.Scatter(
      x = lr_predictions['timestamp'],
      y = lr_predictions['prediction'].astype(float),
      mode = 'lines',
      name = 'Linear Regression predictions'
  )

  trace2 = go.Scatter(
      x = lr_predictions['timestamp'],
      y = lr_predictions['prediction'].astype(float),
      mode = 'lines',
      name = 'Linear Regression predictions'
  )

  trace3 = go.Scatter(
      x = glr_predictions['timestamp'],
      y = glr_predictions['prediction'].astype(float),
      mode = 'lines',
      name = 'Generalized Linear Regression predictions'
  )

  trace4 = go.Scatter(
      x = rf_predictions['timestamp'],
      y = rf_predictions['prediction'].astype(float),
      mode = 'lines',
      name = 'Random Forest Regressor predictions'
  )

  trace5 = go.Scatter(
      x = gbt_predictions['timestamp'],
      y = gbt_predictions['prediction'].astype(float),
      mode = 'lines',
      name = 'GBTRegressor predictions'
  )

  layout = dict(
      title='Test and predictions with Rangeslider',
      xaxis=dict(
          rangeselector=dict(
              buttons=list([
                  #change the count to desired amount of months.
                  dict(count=1,
                      label='1m',
                      step='month',
                      stepmode='backward'),
                  dict(count=6,
                      label='6m',
                      step='month',
                      stepmode='backward'),
                  dict(count=12,
                      label='1y',
                      step='month',
                      stepmode='backward'),
                  dict(count=36,
                      label='3y',
                      step='month',
                      stepmode='backward'),
                  dict(step='all')
              ])
          ),
          rangeslider=dict(
              visible = True
          ),
          type='date'
      )
  )

  data = [trace1,trace2,trace3, trace4, trace5]
  fig = dict(data=data, layout=layout)
  iplot(fig, filename = "Test and predictions with Rangeslider")

In [66]:
show_results(test_df.toPandas(), lr_predictions, glr_predictions, rf_predictions, gbt_predictions)